In [1]:
import sys
import os
import numpy as np 
from keras.utils import to_categorical
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
# from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
import keras.backend as K
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint, Callback
from keras_contrib.layers import CRF
from numpy import newaxis
from random import shuffle
import math
import sklearn
import subprocess
import fastText
import pickle

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def getCasing(word, caseLookup):
    
    if word == 'PADDING_TOKEN':
        return(caseLookup['PADDING_TOKEN'])
    
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
   
    return caseLookup[casing]

In [3]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [4]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [5]:
def get_sentences_germeval(path):
    sentences = []
    with open(path, 'r', encoding = 'UTF-8') as f:
        sentence = []
        for line in f:
            
            line = line.strip()
            
            # append sentence
            if len(line) == 0:
                if len(sentence):
                    sentences.append(sentence)
                sentence = []
                continue
            
            # get sentence tokens
            splits = line.split()
            if splits[0] == '#':
                continue
            temp = [splits[1],splits[2]]
            sentence.append(temp)
        
        # append last
        if len(sentence):
            sentences.append(sentence)    
    return sentences

In [6]:
# preproecessing data from Conll
def get_sentences_conll(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [7]:
trainSentences = get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = get_sentences('../data/CONLL/deu/deu_utf.train')
# devSentences = get_sentences('../data/CONLL/deu/deu_utf.testa')
# testSentences = get_sentences('../data/CONLL/deu/deu_utf.testb')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [8]:
print(testSentences[0])

[['1951', 'O'], ['bis', 'O'], ['1953', 'O'], ['wurde', 'O'], ['der', 'O'], ['nördliche', 'O'], ['Teil', 'O'], ['als', 'O'], ['Jugendburg', 'O'], ['des', 'O'], ['Kolpingwerkes', 'B-OTH'], ['gebaut', 'O'], ['.', 'O']]


In [9]:
labelSet = set()
characters= set()
max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > max_sequence_length:
            max_sequence_length = len(sentence)

In [10]:
print(len(labelSet))
print(max_sequence_length)

25
56


In [11]:
# :: Create a mapping for the labels ::
label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [12]:
print(label2Idx)

{'I-OTHderiv': 1, 'B-ORGpart': 15, 'I-ORGpart': 2, 'B-LOCpart': 3, 'I-LOCpart': 16, 'O': 5, 'I-LOCderiv': 6, 'B-OTHpart': 4, 'I-PERpart': 17, 'I-OTH': 18, 'B-PERpart': 19, 'B-PERderiv': 20, 'I-OTHpart': 22, 'I-PER': 23, 'B-LOC': 7, 'I-LOC': 24, 'B-OTH': 8, 'I-ORG': 9, 'B-LOCderiv': 10, 'B-ORG': 12, 'I-PERderiv': 21, 'I-ORGderiv': 13, 'B-ORGderiv': 25, 'PADDING_TOKEN': 0, 'B-PER': 14, 'B-OTHderiv': 11}


In [13]:
# :: Hard coded case lookup ::
case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [14]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'other': 5, 'numeric': 1, 'mainly_numeric': 6, 'contains_digit': 7, 'PADDING_TOKEN': 0, 'initialUpper': 4, 'allLower': 2, 'allUpper': 3}


In [15]:
print(type(trainSentences))

<class 'list'>


In [16]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [17]:
char2Idx={"PADDING_TOKEN":0}
for char in characters:
    char2Idx[char] = len(char2Idx)
char2Idx['UNKNOWN'] = len(char2Idx)
print(char2Idx)

{'ş': 1, '傳': 2, 'б': 8, 'у': 4, '€': 5, 'υ': 6, 'ь': 7, 'X': 9, 'ċ': 12, '\x94': 13, 'η': 14, '`': 113, 'UNKNOWN': 329, '–': 15, '별': 17, 'h': 3, '\u200e': 18, ']': 57, 'в': 19, '≤': 177, 'ế': 231, 'û': 20, 'ɨ': 10, 'р': 21, '李': 22, '\x80': 23, 'ό': 117, '⊃': 24, '\x92': 219, '©': 179, 'Л': 25, 'ї': 204, 'Ł': 122, 'Æ': 26, '_': 29, '”': 28, 'ņ': 325, 'е': 30, '公': 31, 'ا': 32, 'ă': 309, 'B': 33, 'ü': 35, 'Ġ': 106, 'ж': 268, 'v': 39, 'd': 40, '(': 41, 'И': 47, 'Λ': 44, '}': 45, 'p': 46, 'H': 48, 'ю': 81, '守': 221, 'È': 49, 'A': 50, '鷹': 60, '*': 52, '[': 54, '대': 63, '\x9a': 61, 'w': 56, 'ś': 58, 'л': 184, 'n': 59, 'ب': 284, 'Č': 68, 'ḫ': 64, 'М': 65, 'έ': 66, 'λ': 69, 'Â': 72, 'ο': 71, 'P': 74, 'š': 75, 'ğ': 76, 'ð': 285, 'o': 79, 'н': 80, '柯': 82, 'ç': 251, 'ý': 83, '9': 84, 'ř': 85, '算': 185, 'д': 287, '8': 86, 'K': 88, '4': 89, '§': 283, '‐': 90, 'α': 77, '²': 91, 'г': 92, 'ê': 93, '>': 94, 'ŏ': 95, '›': 96, 'Š': 235, 'ς': 97, 'ъ': 101, 'â': 100, '殿': 102, '´': 103, 'Å': 104, 'ť':

In [18]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [19]:
ft = fastText.load_model("../embeddings/wiki.de.bin")
# ft = fastText.load_model("../embeddings/cc.de.300.bin")

nb_embedding_dims = ft.get_dimension()

In [20]:
print(nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [21]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [22]:
def createBatches(dataset, batch_size):
    batches = []
    temp = []
    i = 0
    for item in dataset:
        temp.append(item)
        i += 1
        if i == batch_size:
            batches.append(temp)
            temp = []
            i = 0
    if len(temp) > 0:
        batches.append(temp)
    return batches

In [23]:
batch_size = 32
# train_batches = createBatches(trainSentences, batch_size)
# dev_batches = createBatches(devSentences, batch_size)
# test_batches = createBatches(testSentences, batch_size)

In [24]:
i = 0
for example_in, example_out in generator(train_batches):
    # [0] word: batch_size * sequence_length * word_embedding_dim
    print(len(example_in[0][31]))
    print(len(example_in[0][31]))
    # [1] case: batch_size * sequence_length * case_embedding_dim

    # [2] char: batch_size * sequence_length * char_embedding_dim
    i += 1
    if i > 9:
        break

NameError: name 'generator' is not defined

In [25]:
def generator(data, shuffle_data = False):
    
    print("Creating batches ...")
    batches = createBatches(data, batch_size)
    
    if shuffle_data:
        print("Shuffling ...")
        shuffle(batches)
    
    while True:
        for batch in batches:
            word_embeddings = []
            case_embeddings = []
            char_embeddings = []
            
            output_labels = []
            
            # batches made according to the size of the sentences. len(batch) gives the size of current batch
            for index in range(len(batch)): 
                sentence = batch[index]
    #             print(sentence)
                
                temp_word= []
                temp_casing = []
                temp_char= []
                
                temp_output=[]
                
                # padding
                words_to_pad = max_sequence_length - len(sentence)
                for i in range(words_to_pad):
                    sentence.append(['PADDING_TOKEN', 'PADDING_TOKEN'])
                
                # create data input for words
                for word in sentence:
                    word, label = word
                    temp_output.append(label2Idx[label])
                    
                    casing = getCasing(word, case2Idx)
                    temp_casing.append(casing)
                        
                    if word == 'PADDING_TOKEN':
                        temp_char2=np.array([char2Idx['PADDING_TOKEN']])
                        temp_char.append(temp_char2)
                        word_vector = [0] * nb_embedding_dims
                        temp_word.append(word_vector)
                    else:
                        # char
                        temp_char2=[]
                        for char in word:
                            if char in char2Idx.keys():
                                temp_char2.append(char2Idx[char])
                            else:
                                temp_char2.append(char2Idx['UNKNOWN']) # To incorporate the words which are not in the vocab
                        temp_char2 = np.array(temp_char2)
                        temp_char.append(temp_char2)
                        
                        # word
                        word_vector = ft.get_word_vector(word.lower())
                        # word_vector = ft.get_word_vector(word)
                        temp_word.append(word_vector)
                        
                temp_char = pad_sequences(temp_char, 52)
                word_embeddings.append(temp_word)
                case_embeddings.append(temp_casing)
                char_embeddings.append(temp_char)
                temp_output = to_categorical(temp_output, len(label2Idx))
                output_labels.append(temp_output)
    #             output_labels = to_categorical()
    #             output_labels = np.array(output_labels)
    #             output_labels = output_labels[...,newaxis]

    #             print(np.array(word_embeddings).shape)
    #             print(np.array(case_embeddings).shape)
    #             print(np.array(char_embeddings).shape)
    #             print(output_labels.shape)
    #             print("******************\n\n")
            yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

def get_label_from_categorical(a):
    labels = []
    for label in a:
        label = np.ndarray.tolist(label)
        label = np.argmax(label)
        labels.append(label)
    return(labels)

def predict_batches(batch):
    steps = 0
    true_labels = []
    pred_labels = []
    for input_data, output_data in generator(batch):
        pred_labels_batch = model.predict(input_data)
        for s in pred_labels_batch:
            pred_labels.append(get_label_from_categorical(s))
        for s in output_data:
            true_labels.append(get_label_from_categorical(s))
        steps += 1
        if steps == math.ceil(len(batch) / batch_size):
            break
    return(true_labels, pred_labels)


def predict_batches_ignore_padding(batch):
    steps = 0
    true_labels = []
    pred_labels = []
    for input_data, output_data in generator(batch):
        
        pred_labels_batch = model.predict(input_data)
        for s_id, s in enumerate(output_data):
            not_padded_true = []
            not_padded_pred = []
            predicted_labels = get_label_from_categorical(pred_labels_batch[s_id])
            for t_id, t in enumerate(get_label_from_categorical(s)):
                if t != 0:
                    not_padded_true.append(t)
                    not_padded_pred.append(predicted_labels[t_id])
            true_labels.append(not_padded_true)
            pred_labels.append(not_padded_pred)
            
        steps += 1
        if steps == math.ceil(len(batch) / batch_size):
            break
    return(true_labels, pred_labels)

In [26]:
idx2Label = {v: k for k, v in label2Idx.items()}
print(len(label2Idx))
print(len(idx2Label))

26
26


In [27]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [28]:
def get_model():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,52,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    kernel_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in kernel_sizes:
        conv = TimeDistributed(Conv1D(
                             kernel_size=sz,
                             filters=32,
                             padding="same",
                             activation="relu",
                             strides=1))(embed_char_out)
        conv = TimeDistributed(MaxPooling1D(52))(conv)
        conv = TimeDistributed(Flatten())(conv)
        conv_blocks.append(conv)
    output = concatenate([words_input, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
    output = TimeDistributed(Dense(len(label2Idx)))(output)
    crf = CRF(len(label2Idx))
    output = crf(output)
    model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

class F1History(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.f1_scores = []
        self.max_f1 = 0

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs.get('val_acc'))
        true_labels, pred_labels = predict_batches_ignore_padding(devSentences)
        pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
        self.f1_scores.append(f1)
        if epoch > -1 and f1 > self.max_f1:
            print("\nNew maximum F1 score: " + str(f1) + " (before: " + str(self.max_f1) + ") Saving to " + tmp_model_filename)
            self.max_f1 = f1
            model.save(tmp_model_filename)

In [ ]:
tmp_model_filename = 'tmp_generator_NER_best.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = F1History()
model = get_model()
model.fit_generator(
    generator(trainSentences, shuffle_data=True), steps_per_epoch = math.ceil(len(trainSentences) / batch_size), 
    validation_data = generator(devSentences), validation_steps = math.ceil(len(devSentences) / batch_size), 
    epochs = 12, callbacks = [history]
)

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = predict_batches_ignore_padding(testSentences)
print(compute_f1(pred_labels, true_labels, idx2Label))

# Experiments

In [29]:
f = open('results_padding.txt', 'w')
for run_i in range(7,10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_generator_NER_padding_best.' + str(run_i) + '.h5'
    # tmp_model_filename = 'tmp_generator_NER_best.h5'
    # checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
    history = F1History()
    
    model = get_model()
    model.fit_generator(
        generator(trainSentences, shuffle_data=True), steps_per_epoch = math.ceil(len(trainSentences) / batch_size), 
        validation_data = generator(devSentences), validation_steps = math.ceil(len(devSentences) / batch_size), 
        epochs = 15, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    true_labels, pred_labels = predict_batches_ignore_padding(testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 7
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10560       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_4 (TimeDistrib (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time

750/750 [==============================] - 248s 330ms/step - loss: -0.2544 - acc: 0.9937 - val_loss: -0.2595 - val_acc: 0.9912
Epoch 13/15
750/750 [==============================] - 247s 329ms/step - loss: -0.2826 - acc: 0.9943 - val_loss: -0.2860 - val_acc: 0.9910
Epoch 14/15
750/750 [==============================] - 252s 335ms/step - loss: -0.3098 - acc: 0.9945 - val_loss: -0.3120 - val_acc: 0.9909
Epoch 15/15
750/750 [==============================] - 249s 332ms/step - loss: -0.3370 - acc: 0.9946 - val_loss: -0.3390 - val_acc: 0.9911
Creating batches ...
Run 8
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (

Epoch 9/15
750/750 [==============================] - 238s 318ms/step - loss: -0.1678 - acc: 0.9931 - val_loss: -0.1745 - val_acc: 0.9909
Epoch 10/15
750/750 [==============================] - 243s 323ms/step - loss: -0.1957 - acc: 0.9936 - val_loss: -0.2011 - val_acc: 0.9909
Epoch 11/15
749/750 [============================>.] - ETA: 0s - loss: -0.2229 - acc: 0.9937Creating batches ...

New maximum F1 score: 0.818941504178273 (before: 0.8173880735649266) Saving to tmp_generator_NER_padding_best.8.h5
750/750 [==============================] - 231s 308ms/step - loss: -0.2229 - acc: 0.9937 - val_loss: -0.2280 - val_acc: 0.9911
Epoch 12/15
749/750 [============================>.] - ETA: 0s - loss: -0.2505 - acc: 0.9941Creating batches ...

New maximum F1 score: 0.8254734496843669 (before: 0.818941504178273) Saving to tmp_generator_NER_padding_best.8.h5
750/750 [==============================] - 224s 299ms/step - loss: -0.2505 - acc: 0.9941 - val_loss: -0.2543 - val_acc: 0.9912
Epoch 13/15

750/750 [==============================] - 240s 320ms/step - loss: -0.1195 - acc: 0.9923 - val_loss: -0.1290 - val_acc: 0.9912
Epoch 8/15
749/750 [============================>.] - ETA: 0s - loss: -0.1474 - acc: 0.9928Creating batches ...

New maximum F1 score: 0.825075075075075 (before: 0.8184534927157266) Saving to tmp_generator_NER_padding_best.9.h5
750/750 [==============================] - 239s 319ms/step - loss: -0.1474 - acc: 0.9928 - val_loss: -0.1562 - val_acc: 0.9915
Epoch 9/15
749/750 [============================>.] - ETA: 0s - loss: -0.1748 - acc: 0.9930Creating batches ...

New maximum F1 score: 0.8259576050576423 (before: 0.825075075075075) Saving to tmp_generator_NER_padding_best.9.h5
750/750 [==============================] - 239s 318ms/step - loss: -0.1748 - acc: 0.9930 - val_loss: -0.1829 - val_acc: 0.9915
Epoch 10/15
750/750 [==============================] - 239s 318ms/step - loss: -0.2027 - acc: 0.9935 - val_loss: -0.2091 - val_acc: 0.9916
Epoch 11/15
750/750 [===

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_generator_NER_best.0.h5', 'final_model_germeval.h5')
with open("final_model_germeval.indexes", "w") as f:
    json.dump([idx2Label, label2Idx, char2Idx, case2Idx], f)

In [ ]:
true_labels, pred_labels = predict_batches(test_batches)
print(compute_f1(pred_labels, true_labels, idx2Label))

In [ ]:
print(pred_labels)

In [ ]:
print(testSentences[len(testSentences)-1])

In [ ]:
print(true_labels[len(testSentences)-1])
print(pred_labels[len(testSentences)-1])

In [ ]:
idx2Label[14]

In [ ]:
print(len(testSentences))
print(len(pred_labels))

In [ ]:
type(train_batches[0])